# 日経平均株価 15分足

### 一つ上のディレクトリをパスに追加

In [71]:
import sys

path_up = ".."
if not path_up in sys.path:
    sys.path.append(path_up)

### ライブラリをここで一括インポート

In [72]:
import datetime

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import yfinance as yf

from funcs.common import get_name_15min_chart, get_name_15min_chart_now

## yfinance による株価データの取得

In [73]:
code = "N225"
symbol = f"^{code}"
ticker = yf.Ticker(symbol)
df = ticker.history(period="1d", interval="15m")
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2025-09-24 09:00:00+09:00,45484.410156,45522.890625,45271.761719,45325.109375,0,0.0,0.0
2025-09-24 09:15:00+09:00,45340.539062,45493.500000,45270.488281,45462.921875,0,0.0,0.0
2025-09-24 09:30:00+09:00,45459.738281,45478.730469,45399.019531,45427.648438,0,0.0,0.0
2025-09-24 09:45:00+09:00,45424.031250,45497.070312,45398.898438,45422.250000,0,0.0,0.0
2025-09-24 10:00:00+09:00,45416.031250,45461.910156,45350.000000,45392.281250,0,0.0,0.0
2025-09-24 10:15:00+09:00,45392.941406,45408.929688,45284.250000,45339.671875,0,0.0,0.0
2025-09-24 10:30:00+09:00,45339.050781,45339.050781,45205.468750,45231.109375,0,0.0,0.0
2025-09-24 10:45:00+09:00,45227.320312,45308.640625,45218.238281,45299.761719,0,0.0,0.0
2025-09-24 11:00:00+09:00,45300.621094,45336.628906,45258.148438,45308.269531,0,0.0,0.0


## 前日終値の取得

In [74]:
# 過去一週間のデータを取得
td = datetime.timedelta(days=7)
dt_first = df.index[0]
dt_prev = dt_first - td
df_last = ticker.history(interval="1d", start=str(dt_prev.date()))
print(df_last["Close"])
n = len(df_last)
lastclose = df_last[n - 2 : n - 1]["Close"].iloc[0]
# lastclose = 44902.27 # 9/16 終値
# 確認のため出力
df_last[n - 2 : n - 1]["Close"]

Date
2025-09-17 00:00:00+09:00    44790.378906
2025-09-18 00:00:00+09:00    45303.429688
2025-09-19 00:00:00+09:00    45045.808594
2025-09-22 00:00:00+09:00    45493.660156
2025-09-24 00:00:00+09:00    45345.871094
Name: Close, dtype: float64


Date
2025-09-22 00:00:00+09:00    45493.660156
Name: Close, dtype: float64

### 当日始値 - 前日終値

In [75]:
delta_open = df.head(1)["Open"].iloc[0] - lastclose
print(f"{delta_open:+.2f}")

-9.25


## ローソク足チャート（小さいサイズ）

In [76]:
FONT_PATH = "../fonts/RictyDiminished-Regular.ttf"
fm.fontManager.addfont(FONT_PATH)

# FontPropertiesオブジェクト生成（名前の取得のため）
font_prop = fm.FontProperties(fname=FONT_PATH)
font_prop.get_name()

plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["font.size"] = 9

fig, ax = plt.subplots(figsize=(3, 2))
mpf.plot(
    df,
    type="candle",
    style="default",
    datetime_format="%H:%M",
    xrotation=0,
    update_width_config=dict(candle_linewidth=0.75),
    ax=ax,
)
ax.axhline(y=lastclose, color="red", linewidth=0.75)
ax.set_ylabel(None)
ax.grid()

if "shortName" in ticker.info:
    ax.set_title(f"{ticker.info['shortName']} ({symbol})")
elif "longName" in ticker.info:
    ax.set_title(f"{ticker.info['longName']} ({symbol})")
else:
    ax.set_title(f"{symbol}")

plt.tight_layout()
plt.savefig(get_name_15min_chart(code, dt_first))
plt.show()